## Лекция 2  BM5    

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [678]:
import collections
import nltk
import re
import csv
import numpy
import time
from math import log
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [679]:
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victoriaregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [680]:
def preprocessing(text):
    normal_forms = []
    tokenizer = RegexpTokenizer(r'\w+')
    
    tokens = [morph.parse(token)[0] for token in tokenizer.tokenize(text)]
    normal_forms = [token.normal_form for token in tokens if token not in russian_stopwords and token.normal_form != 'это']
    preproc_text = ' '.join(normal_forms)
       
    return preproc_text, len(normal_forms)


In [681]:
corpus = []
len_doc = {}
mean_len = []
ids = []
queries = []
docs_id = {}
counted_accuracy = []

with open('/Users/victoriaregina/Downloads/quora_question_pairs_rus.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')

    for line_count, row in enumerate(csv_reader):

        if line_count == 0:
            line_count += 1

        else:

            if line_count <= 1000:
                line_count += 1
                
                docs_id[row[0]] = row[1]
                counted_accuracy.append(row[3])

                preproc_text, text_len  = preprocessing(row[1])
                preproc_query, l = preprocessing(row[2])

                ids.append(row[0])
                corpus.append(preproc_text)
                queries.append(preproc_query)
                len_doc[row[0]] = text_len
                mean_len.append(text_len)

            else:
                break


avlen = numpy.mean(mean_len) #средняя длина документа
N = line_count - 1 #всего документов в коллекции


In [692]:
def bm25(idf, tf, k, b, l_doc, avlen):
    
    score = idf * (tf * (k+1)/(tf + k * (1 - b + b * (l_doc/avlen))))
    
    return score

In [693]:
def bm_matrix_prep(corpus, N, k, b, len_doc, avlen):
    
    idf = {}

    vec = CountVectorizer()
    X = vec.fit_transform(corpus)

    df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=ids)

    d = df.isin([0])

    for word in d.columns:
        sum_docs = N - sum(d[word]) #кол-во документов с этим словом

        idf[word] = log((N - sum_docs + 0.5)/(sum_docs + 0.5))

    df['sum'] = df.sum(axis=1)
    tf_table = df.div(df['sum'], axis=0) 
    tf_table = tf_table.fillna(0)   

    bm25_matrix = numpy.zeros((df.shape[0], df.shape[1])) #create zero matrix

    for doc_id, text in enumerate(corpus):
        
        for word_id, word in enumerate(df.columns):

            try:
          
                score = bm25(idf[word], tf.at[str(doc_id), word], k, b, len_doc[str(doc_id)], avlen)
                bm25_matrix[doc_id, word_id] = float(score)
                
            except KeyError:
                pass
            
        
    return bm25_matrix, df.columns

In [694]:
k = 2.0
b = 0.75

bm25_matrix, columns = bm_matrix_prep(corpus, N, k, b, len_doc, avlen)
bm25_df = pd.DataFrame(bm25_matrix, columns=list(columns), index=ids) #dataframe with bm25

In [699]:
k = 2.0
b = 0

bm15_matrix, columns = bm_matrix_prep(corpus, N, k, b, len_doc, avlen)
bm15_df = pd.DataFrame(bm15_matrix, columns=list(columns), index=ids) #dataframe with bm15

In [700]:
k = 2.0
b = 1

bm11_matrix, columns = bm25_matrix_prep(corpus, N, k, b, len_doc, avlen)
bm11_df = pd.DataFrame(bm11_matrix, columns=list(columns), index=ids) #dataframe with bm25

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

# Подсчет метрики по формуле для каждой пары слово-документ

In [685]:
start_time = time.clock()
scores = {}
for query_id, query in enumerate(queries):
    bm25_score = []
    for i in range(0, len(ids)):
        for q in query.split():
        
            try:
                bm25 = bm25_df.at[str(i), q]
                bm25_score.append(bm25)
           
            except KeyError:
                pass
            
        scores[(query, i)] = numpy.sum(bm25_score)
              
print("Затраченное время для подсчета BM25 для каждой пары слово-документ", "{:g} s".format(time.clock() - start_time))       

Затраченное время для подсчета BM25 для каждой пары слово-документ 513.736 s


# Подсчет метрики через умножение матрицы на вектор

In [686]:
position_of_word = {}

for i, word in enumerate(bm25_df.columns):
    position_of_word[word] = i

In [687]:
def vec_matrix_mult(query_vec, query, matrix):
    query_vec = numpy.zeros((matrix.shape[1], 1))
    
    result = {}
    
    for q in query.split():
        
        try:
            query_vec[position_of_word[q]] = 1
            
        except KeyError:
            pass
        
    for doc_id, bm_25 in enumerate(matrix.values.dot(query_vec)):
        
        result[doc_id] = bm_25[0]
        
    return result

In [688]:
scores = {}
start_time = time.clock()
for query_id, query in enumerate(queries):

    result = vec_matrix_mult(query_vec, query, bm25_df)
    
    scores[query] = sorted(result.items(), key=lambda x: (x[1],x[0]), reverse=True)
    
print("Затраченное время для подсчета BM25", "{:g} s".format(time.clock() - start_time))

Затраченное время для подсчета BM25 5.081 s


# Задание 2


Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [701]:
#query = input('Введите запрос: ')
query = 'рождественские каникулы'
query_t, l = preprocessing(query)

result = vec_matrix_mult(query_vec, query_t, bm25_df)

result = sorted(result.items(), key=lambda x: (x[1],x[0]), reverse=True)

for r in result[:10]:
    
    print('РЕЛЕВАНТНЫЙ ДОКУМЕНТ: '+ docs_id[str(r[0])] +'; БЛИЗОСТЬ ПО BM25 РАВНА', r[1])
    

РЕЛЕВАНТНЫЙ ДОКУМЕНТ: как я могу преобразовать необработанные файлы в jpeg на фотографиях в macbook; БЛИЗОСТЬ ПО BM25 РАВНА 0.0
РЕЛЕВАНТНЫЙ ДОКУМЕНТ: что такое хорошая песня для лирического розыгрыша; БЛИЗОСТЬ ПО BM25 РАВНА 0.0
РЕЛЕВАНТНЫЙ ДОКУМЕНТ: мы могли бы использовать черенковское излучение атмосферы с гамма-лучами или аналогично изображению поверхности планеты отсюда с наземными телескопами; БЛИЗОСТЬ ПО BM25 РАВНА 0.0
РЕЛЕВАНТНЫЙ ДОКУМЕНТ: я и мои подруги личные части коснулись друг друга, она может забеременеть; БЛИЗОСТЬ ПО BM25 РАВНА 0.0
РЕЛЕВАНТНЫЙ ДОКУМЕНТ: которая является лучшими акциями для покупки и продажи ежедневной торговли; БЛИЗОСТЬ ПО BM25 РАВНА 0.0
РЕЛЕВАНТНЫЙ ДОКУМЕНТ: я просто студент, но у меня нет мотивации или вообще не хожу в школу, может кто-нибудь помочь мне бороться с этим; БЛИЗОСТЬ ПО BM25 РАВНА 0.0
РЕЛЕВАНТНЫЙ ДОКУМЕНТ: который 2g-полоса 900 1800mhz была продана во время 2-граммового афера; БЛИЗОСТЬ ПО BM25 РАВНА 0.0
РЕЛЕВАНТНЫЙ ДОКУМЕНТ: у вас есть пара

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

In [702]:
def accuracy_counting(matrix):
    
    scores = {}
    ACCURACY = 0

    for query_id, query in enumerate(queries):
        accuracy = 0

        result = vec_matrix_mult(query_vec, query, matrix)

        scores[query] = sorted(result.items(), key=lambda x: (x[1],x[0]), reverse=True)

        for r in scores[query][:5]:

            if int(counted_accuracy[r[0]]) == 1 and r[1] > 0:
                accuracy += 1

            else:
                accuracy = accuracy

        if accuracy > 0:
            ACCURACY += 1

    return ACCURACY/N

In [703]:
print('Точность поиска при b=0.75 равна', accuracy_counting(bm25_df))
print('Точность поиска при b=0 равна', accuracy_counting(bm15_df))
print('Точность поиска при b=1 равна', accuracy_counting(bm11_df))

Точность поиска при b=0.75 равна 0.648
Точность поиска при b=0 равна 0.648
Точность поиска при b=1 равна 0.648
